In [176]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

def make_sweep_str(pfile, pars, group):
    
    base_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   
    base_str = base_str + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        base_str += ' ' + k + f'={pars[k]}'
        name += k.split('.')[-1]+f':{pars[k]}x'
    name = name[:-1]
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   
    base_str = base_str + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [177]:
group = 'sweep_ff15'
pfile = 'i_psf_max_norm_foci'

variable_col = param_iter()
variable_col.add('training.micro.int_quantile',0.3,0.6,0.9,1.)

par_prod = variable_col.param_product()

# name = 'Ref_SI'

In [178]:
Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=i_psf_max_norm_foci training.micro.int_quantile=0.3 run_name=int_quantile:0.3 output.group=sweep_ff15

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=i_psf_max_norm_foci training.micro.int_quantile=0.6 run_name=int_quantile:0.6 output.group=sweep_ff15

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=i_psf_max_norm_foci training.micro.int_quantile=0.9 run_name=int_quantile:0.9 ou

In [131]:
model_files = sorted(glob.glob('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/Fig_sim_density/sweep_ff6/*'))
out_file = 'data/results/test_file.hdf5'

# with open('run_slurm.sh', 'w') as f:
#     for pars in par_prod:  
#         exec_str = make_sweep_str(pfile, pars, group)
#         print(exec_str)
#         f.write(exec_str)

In [135]:
[Path(m).name for m in model_files]

['int_conc:1.0',
 'int_conc:2.0',
 'int_conc:3.0',
 'int_conc:4.0',
 'int_conc:5.0',
 'int_conc:6.0']